In [34]:
#Use getpass to obtain user netID
import getpass
from os import lseek
import sys

# Import Libraries
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.mllib.evaluation import RankingMetrics
from pyspark import SparkContext,SparkConf

import pandas as pd
import numpy as np
import random
import warnings
import pyspark
import sys

# Import Libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import RankingMetrics
from pyspark import SparkConf, SparkContext
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from math import sqrt

In [91]:
spark = SparkSession.builder.appName('popularity').getOrCreate()
rank = 50
regParam = 0.1
maxIter = 20

ratings_train = spark.read.parquet('train_combined_small_set.parquet')
ratings_val = spark.read.parquet('val_small_set.parquet')


start_time = time.time()
print('maxIter: ', maxIter, 'regParam: ', regParam, 'rank: ', rank)
print('start at: ', start_time)
ratings_train.createOrReplaceTempView('ratings_train')
print("Ratings")

maxIter:  20 regParam:  0.1 rank:  50
start at:  1652806505.033959
Ratings


In [92]:
als = ALS(rank = rank, maxIter=maxIter, regParam=regParam, userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy="drop")
model = als.fit(ratings_train)

In [93]:
ratings_train.createOrReplaceTempView('ratings_train')
ratings_train.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     2|    318|   3.0|1445714835|
|     2|    333|   4.0|1445715029|
|     2|   1704|   4.5|1445715228|
|     2|   3578|   4.0|1445714885|
|     2|   6874|   4.0|1445714952|
|     2|   8798|   3.5|1445714960|
|     2|  46970|   4.0|1445715013|
|     2|  48516|   4.0|1445715064|
|     2|  58559|   4.5|1445715141|
|     2|  60756|   5.0|1445714980|
|     2|  68157|   4.5|1445715154|
|     2|  71535|   3.0|1445714974|
|     2|  74458|   4.0|1445714926|
|     2|  77455|   3.0|1445714941|
|     2|  79132|   4.0|1445714841|
|     2|  80489|   4.5|1445715340|
|     2|  80906|   5.0|1445715172|
|     2|  86345|   4.0|1445715166|
|     2|  89774|   5.0|1445715189|
|     2|  91529|   3.5|1445714891|
+------+-------+------+----------+
only showing top 20 rows



In [94]:
model = als.fit(ratings_train)
# ratings_val = ratings_val_orig
ratings_val.createOrReplaceTempView('ratings_val')
userSubsetRecs = spark.sql('select userId from ratings_val group by userId')
userSubsetRecs.createOrReplaceTempView('userSubsetRecs')

userSubsetRecs.show()

+------+
|userId|
+------+
|    91|
|   140|
|   556|
|   182|
|    41|
|   154|
|   425|
|   381|
|   438|
|   509|
|   288|
|   576|
|   151|
|   414|
|   375|
|   597|
|   368|
|     6|
|    86|
|   169|
+------+
only showing top 20 rows



In [95]:
predicted = model.recommendForUserSubset(userSubsetRecs, 100)

In [96]:
def extractMovieIds(rec):
    return [row.movieId for row in rec]

extractRecMovieIdsUDF = fn.udf(lambda r: extractMovieIds(r), T.ArrayType(T.IntegerType()))
predicted = predicted.select(
    fn.col('userId').alias('pr_userId'),
    extractRecMovieIdsUDF('recommendations').alias('rec_movie_id_indices')
)

In [97]:
file_name = 'val_ALS_small_predicted_' + str(rank) + "_" + str(regParam) + "_" + str(maxIter) + '.parquet'
predicted.write.mode('overwrite').parquet(file_name)